In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import time
import sys
import re
import math
import numpy
import pandas as pd
import xlwt
import random
import os
from os import path

In [5]:
#2. 파일을 가져와서 리스트화 하기
#자정을 넘겨서 하지 말것
now = time.localtime()
date = '%d-%d-%d' % (now.tm_year, now.tm_mon, now.tm_mday)
dir = 'C:/Users/Hyunwoo Park/Desktop/lab/bigdata/e-commerce_crawling/href_crawling_'+date
href = open(dir, 'r').read().split('\n')
len(href)

6292

In [6]:
f_dir = 'C:/Users/Hyunwoo Park/Desktop/lab/bigdata/main_data/'
f_name = f_dir+date+'.csv'
f_name

SyntaxError: invalid syntax (<ipython-input-6-dc19a6c741ac>, line 1)

In [16]:
def parsing(query_url):
    
    path = 'C:/Users/Hyunwoo Park/Desktop/chromedriver/chromedriver.exe'
    driver = webdriver.Chrome(path)
    driver.get(query_url)
    time.sleep(0.1)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    #driver.find_element_by_xpath('''//*[@id="tabMenuDetail2"]''').click()

    product_info = []
    
    #제품명
    try:
        product_name = soup.find('h1', class_ = 'title').get_text().strip()
    except AttributeError:
        product_name = ''
    product_info.append(product_name)
    
    #날짜
    product_info.append(date)
    
    #상품군
    try:
        product_type = soup.find_all('em', class_ = 'selected')
    except AttributeError:
        product_type = ''
    if product_type == []:
        return None
    elif product_type == '':
        return None
    product_large_type = product_type[0].get_text().strip()
    product_medium_type = product_type[1].get_text().strip()
    product_small_type = product_type[2].get_text().strip()
    
    product_info.append(product_large_type)
    product_info.append(product_medium_type)
    product_info.append(product_small_type)
    
    #판매량
    try:
        sales_volume = soup.find('dl', class_ = 'c_product_info_deal')
        sales_volume = sales_volume.find_all('dd')
        sales_volume = sales_volume[1].get_text().replace('\n', '').strip()
        sales_volume = sales_volume[:12]
    except AttributeError:
        sales_volume = ''
    product_info.append(sales_volume)
    
    #남은 세일 시간
    try:
        sales_time = soup.find('dd', class_ = 'period').get_text()
    except AttributeError:
        sales_time = ''
    if sales_time == '':
        return None

    product_info.append(sales_time)
    
    #무료배송
    try:
        free_delivery = soup.find('button', 'c_product_btn_more5').find('strong').get_text().strip()
    except AttributeError:
        free_delivery = ''
    if free_delivery == '':
        try:
            free_delivery = soup.find('div', 'delivery').find('dt').get_text().strip()
        except AttributeError:    
            free_delivery = ''
    product_info.append(free_delivery)
    
    #리뷰 수
    try: 
        review_num = soup.select('a[data-log-actionid-label=review_num]')
        if review_num == []:
            review_num = 0
        else:
            review_num = soup.select('a[data-log-actionid-label=review_num]')[0].get_text().replace("\n",'').strip()
            review_num = review_num.replace(' 리뷰 보기', '')
    except AttributeError:
        review_num = ''

    
    product_info.append(review_num)
    
    #가격
    #할인 퍼센트
    try:
        price_info = soup.find_all('span', class_ = 'value')
        discount_per = price_info[0].get_text().strip()
        product_price = price_info[1].get_text().strip()
    except AttributeError:
        price_info = ''
        discount_per = ''
    if price_info == []:
        return None
    elif price_info == '':
        return None
    product_info.append(product_price)
    product_info.append(discount_per)
    
    #공급자 이름
    try: 
        supplier_name = soup.select('a[data-log-actionid-label=store_go]')[0].get_text().replace("\n",'').strip()
    except AttributeError:
        supplier_name = ''
    product_info.append(supplier_name)
    
    #공식인증점
    try:
        official_store = soup.find('dd', class_ = 'flag_official').get_text().strip()
    except AttributeError:
        official_store = ''
    product_info.append(official_store)
    
    #고객서비스 우수
    try:
        customer_service = soup.find('dd', class_ = 'flag_service').get_text().strip()
    except AttributeError:
        customer_service = ''
    product_info.append(customer_service)
    
    #공급자 긍정리뷰 퍼센트
    try:
        supplier_info = soup.find_all('dd', class_ = 'text_num')
    except AttributeError:
        supplier_info = ''
    global supplier_pos_review_per
    global supplier_response_per
    if not supplier_pos_review_per == '':
        supplier_pos_review_per = supplier_info[0].get_text().strip()
        supplier_response_per = supplier_info[1].get_text().strip()
    product_info.append(supplier_pos_review_per)
    product_info.append(supplier_response_per)
    
    #공급자 판매량
    try:
        supplier_sales = soup.find('span', class_ = 'c_product_store_rate').get_text().strip()
    except AttributeError:
        supplier_sales = ''
    if not supplier_sales == '':
        supplier_sales = supplier_sales.replace('총 5점 중\n', '')
        supplier_sales = supplier_sales.replace('점', '')
    product_info.append(supplier_sales)
    
    #상품리뷰 클릭
    
    #driver.find_element_by_xpath('''//*[@id="tabMenuDetail2"]''').click()
    
    driver.switch_to.frame('ifrmReview')
    html2 = driver.page_source
    soup2 = BeautifulSoup(html2, 'html.parser')
    
    #전체 평점
    try:
        total_satis = soup2.find('span', class_ = 'c_seller_grade').get_text().strip()
    except AttributeError:
        total_satis = ''

    if not total_satis == '':    
        total_satis = total_satis.replace('판매자 평점 별5개 중 ', '')
        total_satis = total_satis.replace('개', '')
        total_satis = total_satis.replace('판매자 평점 5점 중 ', '')
    product_info.append(total_satis)
    
    #평점분포
    try:
        star_distr = soup2.find('dl', class_ = 'c_product_review_rate2').find_all('span', 'rate')
        five_star_per = star_distr[0].get_text().strip()
        four_star_per = star_distr[1].get_text().strip()
        three_star_per = star_distr[2].get_text().strip()
        two_star_per = star_distr[3].get_text().strip()
        one_star_per = star_distr[4].get_text().strip()
    except:
        five_star_per = ''
        four_star_per = ''
        three_star_per = ''
        two_star_per = ''
        one_star_per = ''
    
    product_info.append(five_star_per)
    product_info.append(four_star_per)
    product_info.append(three_star_per)
    product_info.append(two_star_per)
    product_info.append(one_star_per)
    
    #기능 별 의견은 나중에 preprocessing 필요
    global functional_quality_name
    global functional_quality
    global functional_quality_opinion2
    try:
        functional_quality = soup2.find('dl', class_ = 'c_product_review_rate3')
        functional_quality_name = functional_quality.find_all('dt')
        functional_quality_opinion = functional_quality.find_all('dd')
        for i in range(0, len(functional_quality_name)):
            functional_quality_name[i] = functional_quality_name[i].get_text().strip()

        functional_quality_per = []
        functional_quality_opinion2 = []
        for i in range(0, len(functional_quality_name)):
            functional_quality_per.append(functional_quality_opinion[2*i - 1])
        for i in range(0, len(functional_quality_name)):
            functional_quality_per[i] = functional_quality_per[i].get_text().strip()
        for i in range(0, len(functional_quality_name)):
            functional_quality_opinion2.append(functional_quality_opinion[2*i])
        for i in range(0, len(functional_quality_name)):
            functional_quality_opinion2[i] = functional_quality_opinion2[i].get_text().strip()

    except AttributeError:
        funtional_quality_name = ''
        functional_quality_per = ''
        fucntional_quality_opinion2 = ''
    
    product_info.append(functional_quality_name)
    product_info.append(functional_quality_per)
    product_info.append(functional_quality_opinion2)
    
    #주제별 리뷰 수
    try:
        review_content = soup2.find_all('label', 'c_product_check')
        review_content2 = []
        if review_content == []:
            pass
        else:
            for i in range(0, len(review_content)):
                review_content2.append(review_content[i].find('span').get_text())
            review_content2.remove('전체')
    except AttributeError:
        review_content2 = ''
    
    product_info.append(review_content2)
    
    driver.close()
    
    return product_info

,a,b,c,d,e
0,1,2,3,4,5
1,6,7,8,9,10


In [ ]:
daily_info = pd.DataFrame(columns = ['product name', 'date', 'product_large_type','product_medium_type', 'product_small_type',
                                    'total_sales', 'remain_sales_time', 'free_delivery', 'review_num', 'price', 'discount_per',
                                     'supplier_name', 'official_store', 'customer_service', 'supplier_pos_review', 'supplier_response_per',
                                    'supplier_sales_performance', 'customer_total_satisfaction', 'five_star_per', 
                                     'four_star_per', 'three_star_per', 'two_star_per', 'one_star_per', 'functional_quality_name', 'functional_quality_per',
                                    'functional_quality_opinion', 'review_content' ])
for i in range(40, 50):
    query_url = href[i]
    total_info = parsing(query_url)
    daily_info = daily_info.append(pd.Series(total_info, index = daily_info.columns), ignore_index = True)
    print(i)
    
daily_info.to_csv(f_name, encoding = 'utf-8-sig')
